In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
#Train
df_tr = pd.read_csv('/content/Re-AAC+APAAC.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
X = df_tr[columns]
Y = df_tr[target]

In [ ]:
# #Test
# df_ts = pd.read_csv('/content/Re-AAC+APAAC_Test.csv')
# columns = df_ts.columns.tolist()
# # Filter the columns to remove data we do not want
# columns = [c for c in columns if c not in ["Target"]]
# # Store the variable we are predicting
# target = "Target"
# xtest = df_ts[columns]
# ytest = df_ts[target]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.3)

In [ ]:
total_Metics_cv_test = []
total_Metics_cv_test = pd.DataFrame(total_Metics_cv_test)
total_Metics_cv_test['Classifier'] = 'Classifier'
total_Metics_cv_test['Accuracy'] = 'Accuracy'
total_Metics_cv_test['mcc'] = 'mcc'
# total_Metics_cv_test['auc'] = 'auc'
total_Metics_cv_test['Kappa'] = 'Kappa'
total_Metics_cv_test['precision'] = 'precision'
total_Metics_cv_test['recall'] = 'recall'
total_Metics_cv_test['f1'] = 'f1'
total_Metics_cv_test['sensitivity'] = 'sensitivity'
total_Metics_cv_test['specificity'] = 'specificity'

cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
total_Metics_ind_test = []
total_Metics_ind_test = pd.DataFrame(total_Metics_ind_test)
total_Metics_ind_test['Classifier'] = 'Classifier'
total_Metics_ind_test['Accuracy'] = 'Accuracy'
total_Metics_ind_test['mcc'] = 'mcc'
# total_Metics_ind_test['auc'] = 'auc'
total_Metics_ind_test['Kappa'] = 'Kappa'
total_Metics_ind_test['precision'] = 'precision'
total_Metics_ind_test['recall'] = 'recall'
total_Metics_ind_test['f1'] = 'f1'
total_Metics_ind_test['sensitivity'] = 'sensitivity'
total_Metics_ind_test['specificity'] = 'specificity'

In [ ]:
# create model
models = [RandomForestClassifier(n_estimators = 400, max_depth = 19),
          XGBClassifier(n_estimators = 400,max_depth = 3, learning_rate = 0.2),
          LGBMClassifier(learning_rate = 0.2,max_depth = 15,random_state = 42),
          SVC(kernel='linear', degree=3),
          DecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,  max_features=1, random_state=42),
          LogisticRegression(C=10.0, solver='liblinear', random_state=43),
          MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, activation='relu', solver='adam', random_state=50, learning_rate_init=0.01),
          GradientBoostingClassifier(n_estimators = 400, learning_rate = 0.1, random_state = 42),
          AdaBoostClassifier(n_estimators = 500, learning_rate = 0.1, random_state = 42)]

In [ ]:
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  total_Metics_cv_test.loc[len(total_Metics_cv_test.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

In [ ]:
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  model.fit(xtrain,ytrain)
  pred = model.predict(xtest)
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  total_Metics_ind_test.loc[len(total_Metics_ind_test.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

[LightGBM] [Info] Number of positive: 319, number of negative: 368
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000735 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9779
[LightGBM] [Info] Number of data points in the train set: 687, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.464338 -> initscore=-0.142892
[LightGBM] [Info] Start training from score -0.142892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [ ]:
print(total_Metics_cv_test)

                                          Classifier  Accuracy       mcc  \
0  (DecisionTreeClassifier(max_depth=19, max_feat...  0.941776  0.882956   
1  XGBClassifier(base_score=None, booster=None, c...  0.931587  0.862855   
2  LGBMClassifier(learning_rate=0.2, max_depth=15...  0.946143  0.891943   
3                               SVC(kernel='linear')  0.780204  0.569736   
4  DecisionTreeClassifier(max_features=1, random_...  0.880640  0.759973   
5  LogisticRegression(C=10.0, random_state=43, so...  0.788937  0.584049   
6  MLPClassifier(hidden_layer_sizes=(100, 50), le...  0.836972  0.675984   
7  ([DecisionTreeRegressor(criterion='friedman_ms...  0.924309  0.848028   
8  (DecisionTreeClassifier(max_depth=1, random_st...  0.903930  0.807243   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.882956   0.937304  0.937304  0.937304     0.945652     0.937304  
1  0.862674   0.917178  0.937304  0.927132     0.926630     0.937304  
2  0.891847   0.935185  0.

In [ ]:
print(total_Metics_ind_test)

                                          Classifier  Accuracy       mcc  \
0  (DecisionTreeClassifier(max_depth=19, max_feat...  0.979661  0.958765   
1  XGBClassifier(base_score=None, booster=None, c...  0.979661  0.958765   
2  LGBMClassifier(learning_rate=0.2, max_depth=15...  0.983051  0.965553   
3                               SVC(kernel='linear')  0.789831  0.589137   
4  DecisionTreeClassifier(max_features=1, random_...  0.840678  0.680835   
5  LogisticRegression(C=10.0, random_state=43, so...  0.816949  0.632234   
6  MLPClassifier(hidden_layer_sizes=(100, 50), le...  0.881356  0.763048   
7  ([DecisionTreeRegressor(criterion='friedman_ms...  0.979661  0.958765   
8  (DecisionTreeClassifier(max_depth=1, random_st...  0.955932  0.910736   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.958674   0.969231  0.984375  0.976744     0.976048     0.984375  
1  0.958674   0.969231  0.984375  0.976744     0.976048     0.984375  
2  0.965530   0.976744  0.

In [ ]:
ensemble = []

In [ ]:
model1 = RandomForestClassifier(n_estimators = 400, max_depth = 19)
ensemble.append(('RandomForest',model1))

In [ ]:
model2 = XGBClassifier(n_estimators = 200,max_depth = 3, learning_rate = 0.22)
ensemble.append(('XGBClassifier',model2))

In [ ]:
model3 = LGBMClassifier(learning_rate = 0.2,max_depth = 15,random_state = 42)
ensemble.append(('LGBMClassifier',model3))

In [ ]:
model4 = GradientBoostingClassifier(n_estimators = 400, learning_rate = 0.1, random_state = 42)
ensemble.append(('GradientBoostingClassifier',model4))

In [ ]:
model5 = AdaBoostClassifier(n_estimators = 500, learning_rate = 0.1, random_state = 42)
ensemble.append(('AdaBoost',model5))

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
new_ensemble = VotingClassifier(ensemble)

In [ ]:
new_ensemble.fit(xtrain,ytrain)

[LightGBM] [Info] Number of positive: 319, number of negative: 368
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000691 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9779
[LightGBM] [Info] Number of data points in the train set: 687, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.464338 -> initscore=-0.142892
[LightGBM] [Info] Start training from score -0.142892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

VotingClassifier(estimators=[('RandomForest',
                              RandomForestClassifier(max_depth=19,
                                                     n_estimators=400)),
                             ('XGBClassifier',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=No...
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('LGBMClassifier',
                              LGBMClassifier(learning_rate=0.2, max_depth=15,
                                             random_state=42)),
                             ('GradientBoostingClassifier',
                              GradientBoostingClassifier(n_estimators=400,
                                                         random_state=42)),
                             ('AdaBoost',
                              AdaBoostClassifier(learning_rate=0.1,
                                                 n_estimators=500,
                                                 random_state=42))])

In [ ]:
new_ensemble.score(xtest,ytest)

0.9796610169491525

In [ ]:
h_pred = cross_val_predict(new_ensemble, xtrain, ytrain, cv=cv, n_jobs=-1)
h_Accuracy = accuracy_score(ytrain, h_pred)

In [ ]:
h_Accuracy

0.9388646288209607

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
estimator4 = [('RF', RandomForestClassifier(n_estimators = 100, max_depth = 10)),
             ('XGB', XGBClassifier(n_estimators = 400,max_depth = 3, learning_rate = 0.05)),
              ('LGBM', LGBMClassifier(learning_rate = 0.1,max_depth = 3,random_state = 42)),
             ('GDC', GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.5, random_state = 50)),
             ('ADB', AdaBoostClassifier(n_estimators = 500, learning_rate = 0.1, random_state = 42)),]
Stacking4 = StackingClassifier( estimators=estimator4, final_estimator= LogisticRegression(C=10.0, solver='liblinear', random_state=43))

In [ ]:
Stacking4.fit(xtrain,ytrain)

[LightGBM] [Info] Number of positive: 319, number of negative: 368
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000672 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9779
[LightGBM] [Info] Number of data points in the train set: 687, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.464338 -> initscore=-0.142892
[LightGBM] [Info] Start training from score -0.142892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

StackingClassifier(estimators=[('RF', RandomForestClassifier(max_depth=10)),
                               ('XGB',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type...
                                              n_estimators=400, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('LGBM',
                                LGBMClassifier(max_depth=3, random_state=42)),
                               ('GDC',
                                GradientBoostingClassifier(learning_rate=0.5,
                                                           n_estimators=500,
                                                           random_state=50)),
                               ('ADB',
                                AdaBoostClassifier(learning_rate=0.1,
                                                   n_estimators=500,
                                                   random_state=42))],
                   final_estimator=LogisticRegression(C=10.0, random_state=43,
                                                      solver='liblinear'))

In [ ]:
Stacking4.score(xtest,ytest)

0.9864406779661017

In [ ]:
st4_h_pred = cross_val_predict(Stacking4, xtrain, ytrain, cv=cv, n_jobs=-1)
st4_h_Accuracy = accuracy_score(ytrain, st4_h_pred)

In [ ]:
st4_h_Accuracy

0.9374090247452693

In [ ]:
stack_cv = []
stack_cv = pd.DataFrame(stack_cv)
stack_cv['Classifier'] = 'Classifier'
stack_cv['mcc'] = 'mcc'
# stack_cv['auc'] = 'auc'
stack_cv['Kappa'] = 'Kappa'
stack_cv['precision'] = 'precision'
stack_cv['recall'] = 'recall'
stack_cv['f1'] = 'f1'
stack_cv['sensitivity'] = 'sensitivity'
stack_cv['specificity'] = 'specificity'

cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
stack_ind = []
stack_ind = pd.DataFrame(stack_ind)
stack_ind['Classifier'] = 'Classifier'
stack_ind['mcc'] = 'mcc'
# stack_ind['auc'] = 'auc'
stack_ind['Kappa'] = 'Kappa'
stack_ind['precision'] = 'precision'
stack_ind['recall'] = 'recall'
stack_ind['f1'] = 'f1'
stack_ind['sensitivity'] = 'sensitivity'
stack_ind['specificity'] = 'specificity'

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
pred = cross_val_predict(Stacking4, xtrain, ytrain, cv=cv, n_jobs=-1)
mcc = matthews_corrcoef(ytrain, pred)
cm1 = confusion_matrix(ytrain, pred)
kappa = cohen_kappa_score(ytrain, pred)
f1 = f1_score(ytrain, pred)
precision_score = precision_score(ytrain, pred)
recall_score = recall_score(ytrain, pred)
sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
stack_cv.loc[len(stack_cv.index)] = [Stacking4, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
Stacking4.fit(xtrain,ytrain)
pred = Stacking4.predict(xtest)
mcc = matthews_corrcoef(ytest, pred)
cm1 = confusion_matrix(ytest, pred)
kappa = cohen_kappa_score(ytest, pred)
f1 = f1_score(ytest, pred)
precision_score = precision_score(ytest, pred)
recall_score = recall_score(ytest, pred)
sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
stack_ind.loc[len(stack_ind.index)] = [Stacking4, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

[LightGBM] [Info] Number of positive: 319, number of negative: 368
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9779
[LightGBM] [Info] Number of data points in the train set: 687, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.464338 -> initscore=-0.142892
[LightGBM] [Info] Start training from score -0.142892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [ ]:
print(stack_cv)

                                          Classifier       mcc     Kappa  \
0  StackingClassifier(estimators=[('RF', RandomFo...  0.868442  0.868408   

   precision    recall        f1  sensitivity  specificity  
0   0.925466  0.934169  0.929797     0.934783     0.934169  


In [ ]:
print(stack_ind)

                                          Classifier       mcc     Kappa  \
0  StackingClassifier(estimators=[('RF', RandomFo...  0.972399  0.972399   

   precision    recall        f1  sensitivity  specificity  
0   0.984375  0.984375  0.984375     0.988024     0.984375  
